In [14]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
! pip install datasets
! pip install torch

In [15]:
! pip install fasttext

In [16]:
import numpy as np
from transformers import BitsAndBytesConfig, AutoConfig, AutoModelForCausalLM, AutoModelWithLMHead
from transformers import GenerationConfig
import torch
from accelerate import init_empty_weights, infer_auto_device_map
import re
from functools import reduce

In [ ]:
from datasets import load_dataset
dataset_1 = load_dataset("codeparrot/xlcost-text-to-code", "Python-program-level")

In [ ]:
dataset_1

In [ ]:
print(dataset_1['train'][24]['text'])

In [ ]:
print(dataset_1['train'][24]['code'])

In [ ]:
# special tokens for prompting
system_token = "<SYSTEM_TASK:>"
user_token = "<USER_TASK:>"

In [ ]:
def process_1(data):
    text_prompts = list(map(lambda x: x.replace(' | ', ','), data['text']))
    text_prompts = list(map(lambda x: x.replace(' \n ', ','), data['text']))
    prompt = ''
    final_code = ''
    final_text_prompt = []
    final_code_prompt = []
    code = data['code']
    for i in range(len(text_prompts)):
        prompt =  text_prompts[i]
        new_prompt = ''
        arr = prompt.split(';')
        for i in range(len(arr)):
# 2
#             new_prompt = new_prompt + 'def ' + arr[i] + '\b():' + '\n\t' + f'"""{arr[i]}"""' + '\n'
# 1
#             new_prompt = new_prompt + 'def' + ' python' + '():' + '\n\t' + f'"""{arr[i]}"""' + '\n'
# 4
#             new_prompt = new_prompt + 'def' + ' write_python_code' + '():' + '\n\t' + f'"""\n\t{arr[i]}\n\t"""' + '\n\n'
# 5
#             new_prompt = new_prompt + 'def' + ' write_python_code' + '():' + '\n\t' + f'"""{arr[i]}"""' + '\n\n'   
# 3
            new_prompt = new_prompt + 'def' + arr[i] + '():' + '\n\t' + f'#{arr[i]}' + '\n'

        code[i] = code[i].replace('NEW_LINE INDENT' , '\n\t')
        code[i] = code[i].replace('NEW_LINE' , '\n')
        code[i] = code[i].replace('DEDENT' , '\b')
        code[i] = code[i].replace(' ( ' , '(')
        code[i] = code[i].replace(' [ ', '[')
        code[i] = code[i].replace('_ ', '_')
        code[i] = code[i].replace('" ', '"')
        final_code = code[i]
        final_text_prompt.append(new_prompt)
        final_code_prompt.append(final_code)
    return {
      'text_prompt' : final_text_prompt,
      'code_prompt' : final_code_prompt
    }

In [ ]:
final_text_prompt = []
final_code_prompt = []
prompted_data_1 = dataset_1.map(process_1, batched = True, remove_columns = ['text', 'code'])

In [ ]:
prompted_data_1

In [ ]:
print(prompted_data_1['train'][4]['text_prompt'])

In [ ]:
print(prompted_data_1['train'][4]['code_prompt'])

In [ ]:
code_lines = prompted_data_1['train'][10]['code_prompt'].split('\n')
func = ''
for i in range(len(code_lines)):
    if 'def' in code_lines[i][:10]:
        func = code_lines[i] + '\n'
        
print(func)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

# special tokens for prompting
system_token = "<SYSTEM_TASK:>"
user_token = "<USER_TASK:>"
assistant_token = "<ASSISTANT_TASK:>"
end_token = "<END_TASK>"


tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot",
                                          additional_special_tokens = ["<SYSTEM_TASK:>", "<USER_TASK:>", "<ASSISTANT_TASK:>", "<END_TASK>"],
                                          pad_token = "<PAD>",
                                          )

In [ ]:
tokenizer

In [ ]:
special_token_dict = tokenizer.special_tokens_map
print(special_token_dict)

In [ ]:
tokenizer.add_special_tokens(special_token_dict)

In [ ]:
context_length = 10000

In [ ]:
config = AutoConfig.from_pretrained("codeparrot/codeparrot",
                                    vocab_size=tokenizer.vocab_size,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
)

with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)
    
print(model)

In [ ]:
device_map = infer_auto_device_map(model, no_split_module_classes = ['GPT2Block'])
print(device_map)

In [ ]:
import torch 

nf4_config = BitsAndBytesConfig(
   load_in_2bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)


model_id = "codeparrot/codeparrot"
model =AutoModelForCausalLM.from_pretrained(model_id,
                                            config = config,
                                            device_map="auto",
                                            quantization_config=nf4_config,
                                            torch_dtype=torch.bfloat16)

In [ ]:
model.resize_token_embeddings(len(tokenizer))

In [ ]:
add_prompt = '<SYSTEM_TASK:>\nGiven the following code description, write Python code to implement the functionality described below\n<END_TASK>\n<USER_TASK:>\nDescription:\n'
exp = prompted_data_1['train'][23]['text_prompt']
exp1 = 'Write a pyhton code for each function as defined in comments\n'
# final_prompt = add_prompt + exp + '\n' + 'def main(): \n\t'
final_prompt = add_prompt + exp1 + exp + '\n'
# +  func + f'"""\n{exp1}\n"""'
print(final_prompt)

In [ ]:
inputs = tokenizer(final_prompt, return_tensors = 'pt').to('cuda')
generation_config = GenerationConfig(max_new_tokens=150, temperature= 1.2, do_sample=True)

output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 200,
        generation_config=generation_config,
        pad_token_id = tokenizer.pad_token_id,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{final_prompt}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

In [ ]:
def dataset_formation(data):
    add_prompt = '<SYSTEM_TASK:>\nGiven the following code description, write Python code to implement the functionality described below\n<END_TASK>\n<USER_TASK:>\nDescription:\n'
    exp = prompted_data_1['train']['text_prompt']
    final_prompt = add_prompt + func + '\t' f'"""{exp}"""'
    code = prompted_data_1['train']['text_prompt']

In [ ]:
add_prompt = '<SYSTEM_TASK:>\nGiven the following code description, write Python code to implement the functionality described below\n<END_TASK>\n<USER_TASK:>\nDescription:\n'

text_prompt = 'def VGG16_model_architecture()\n"""\nInitialize VGG16 model a deep learning model trained on imagenet for performing Image Classification in the VGG16_model_architecture function\n"""\n\bdef train_test_split():\n\t"""Split the data X, Y in to train and test data using sklearn"""\n\bdef model_predict():\n\t"""Make prediction using the deep learning model defined above in VGG16_model_architecture function"""\n'

final_prompt = add_prompt + text_prompt
print(final_prompt)

In [1]:
! pip install git+https://github.com/huggingface/datasets#egg=datasets

  Cloning https://github.com/huggingface/datasets to /tmp/pip-install-5q8y4mj7/datasets_44d6e0a3e76f4a9eadefbd640a3c9d52
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets /tmp/pip-install-5q8y4mj7/datasets_44d6e0a3e76f4a9eadefbd640a3c9d52
  Resolved https://github.com/huggingface/datasets to commit 029227a116c14720afca71b9b22e78eb2a1c09a6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for datasets: filename=datasets-2.14.4.dev0-py3-none-any.whl size=493133 sha256=832f585e84a398cd6d0ff3ef842f201d1a67d0530341a1f491411f86b4fba1f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-tg1a4xxv/wheels/7f/ba/ce/8f6a52388a9966c7d9afa987113a763f7c105f568f369adbc6
Successfully built datasets
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [2]:
from datasets import load_dataset
dataset_2 = load_dataset("code_x_glue_ct_code_to_text", "python", download_mode="force_redownload")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

In [8]:
dataset_2

DatasetDict({
    train: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 251820
    })
    validation: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 13914
    })
    test: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 14918
    })
})

In [9]:
print(dataset_2['train'][5645]['code'])

def check_table_formats(files):
    """
    Determine whether a list of files are of a recognizable output type.

    Parameters
    ----------
    files : str
        A list of file names

    Returns
    -------
    result : bool
        True if *all* the file names are supported
    """
    cont = True
    formats = get_table_formats()
    for t in files.split(','):
        _, ext = os.path.splitext(t)
        ext = ext[1:].lower()
        if ext not in formats:
            cont = False
            log.warn("Format not supported for {0} ({1})".format(t, ext))
    if not cont:
        log.error("Invalid table format specified.")
    return cont


In [10]:
print(dataset_2['train'][43]['docstring'])

Mark a callable as exclusive

    :param via: factory for a Lock to guard the callable

    Guards the callable against being entered again before completion.
    Explicitly raises a :py:exc:`RuntimeError` on violation.

    :note: If applied to a method, it is exclusive across all instances.


In [11]:
def dataset_formation(data):
    doc = data['docstring'].split('\n')
#     for i in range()
    add_prompt = f'<SYSTEM_TASK:>\n{doc[0]}\n<END_TASK>\n<USER_TASK:>\nDescription:\n'
    s = data['code']
    final_prompt = ''
    code = ''
    occurrences = re.finditer('"""', s)
    # using reduce() to get start indices of all occurrences
    res = reduce(lambda x, y: x + [y.start()], occurrences, [])
    if len(res) > 1:     
        pos = res[1]
        prompt = data['code'][:pos+3]
        final_prompt = add_prompt +prompt
        code = s.replace(prompt, '')
    return{
        'text_prompt' : final_prompt,
        'code_prompt' : code
    }

In [17]:
prompted_dataset_2 = dataset_2.map(dataset_formation, batched = False, remove_columns = dataset_2['train'].column_names)

Map:   0%|          | 0/251820 [00:00<?, ? examples/s]

Map:   0%|          | 0/13914 [00:00<?, ? examples/s]

Map:   0%|          | 0/14918 [00:00<?, ? examples/s]

In [18]:
prompted_dataset_2

DatasetDict({
    train: Dataset({
        features: ['text_prompt', 'code_prompt'],
        num_rows: 251820
    })
    validation: Dataset({
        features: ['text_prompt', 'code_prompt'],
        num_rows: 13914
    })
    test: Dataset({
        features: ['text_prompt', 'code_prompt'],
        num_rows: 14918
    })
})

In [19]:
print(prompted_dataset_2['train'][343]['text_prompt'])

<SYSTEM_TASK:>
Check if given regex is of type ECMA 262 or not.
<END_TASK>
<USER_TASK:>
Description:
def is_ecma_regex(regex):
    """Check if given regex is of type ECMA 262 or not.

    :rtype: bool

    """


In [20]:
print(prompted_dataset_2['train'][343]['code_prompt'])


    parts = regex.split('/')

    if len(parts) == 1:
        return False

    if len(parts) < 3:
        raise ValueError('Given regex isn\'t ECMA regex nor Python regex.')
    parts.pop()
    parts.append('')

    raw_regex = '/'.join(parts)
    if raw_regex.startswith('/') and raw_regex.endswith('/'):
        return True
    return False


In [ ]:
final_prompt = prompted_dataset_2['train'][374]['text_prompt']
code = prompted_dataset_2['train'][374]['code_prompt']
inputs = tokenizer(final_prompt, return_tensors = 'pt').to('cuda')
generation_config = GenerationConfig(max_new_tokens=150, temperature= 0.5, do_sample=True, top_p = 1)

output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 200,
        generation_config=generation_config,
        pad_token_id = tokenizer.pad_token_id,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{final_prompt}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN WRITTEN CODE:\n{code}')

**The above dataset is working GOOD**

In [21]:
from datasets import load_dataset
dataset_3 = load_dataset("openai_humaneval")

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

In [22]:
dataset_3

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})

In [23]:
print(dataset_3['test'][50]['prompt'])



def encode_shift(s: str):
    """
    returns encoded string by shifting every character by 5 in the alphabet.
    """
    return "".join([chr(((ord(ch) + 5 - ord("a")) % 26) + ord("a")) for ch in s])


def decode_shift(s: str):
    """
    takes as input string encoded with encode_shift function. Returns decoded string.
    """



In [24]:
print(dataset_3['test'][10]['canonical_solution'])

    if not string:
        return ''

    beginning_of_suffix = 0

    while not is_palindrome(string[beginning_of_suffix:]):
        beginning_of_suffix += 1

    return string + string[:beginning_of_suffix][::-1]



In [25]:
def dataset_formation(data):
    add_prompt = '<SYSTEM_TASK:>\nGiven the following code description, write Python code to implement the functionality described below\n<END_TASK>\n<USER_TASK:>\nDescription:\n'
    prompt = data['prompt']
    final_prompt = add_prompt + prompt
    code = data['canonical_solution']
    return{
        'text_prompt' : final_prompt,
        'code_prompt' : code
    }

In [26]:
prompted_dataset_3 = dataset_3.map(dataset_formation, batched = False, remove_columns = dataset_3['test'].column_names)

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

In [27]:
prompted_dataset_3

DatasetDict({
    test: Dataset({
        features: ['text_prompt', 'code_prompt'],
        num_rows: 164
    })
})

In [28]:
print(prompted_dataset_3['test'][34]['text_prompt'])

<SYSTEM_TASK:>
Given the following code description, write Python code to implement the functionality described below
<END_TASK>
<USER_TASK:>
Description:


def unique(l: list):
    """Return sorted unique elements in a list
    >>> unique([5, 3, 5, 2, 3, 3, 9, 0, 123])
    [0, 2, 3, 5, 9, 123]
    """



In [29]:
print(prompted_dataset_3['test'][34]['code_prompt'])

    return sorted(list(set(l)))



In [30]:
code = prompted_dataset_3['test'][5]['text_prompt']
print(code)

<SYSTEM_TASK:>
Given the following code description, write Python code to implement the functionality described below
<END_TASK>
<USER_TASK:>
Description:
from typing import List


def intersperse(numbers: List[int], delimeter: int) -> List[int]:
    """ Insert a number 'delimeter' between every two consecutive elements of input list `numbers'
    >>> intersperse([], 4)
    []
    >>> intersperse([1, 2, 3], 4)
    [1, 4, 2, 4, 3]
    """



In [ ]:
final_prompt = prompted_dataset_3['test'][74]['text_prompt']
code = prompted_dataset_3['test'][74]['code_prompt']
inputs = tokenizer(final_prompt, return_tensors = 'pt').to('cuda')
generation_config = GenerationConfig(max_new_tokens=150, temperature= 0.1, do_sample=True, top_p = 4)

output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 100,
        generation_config=generation_config,
        pad_token_id = tokenizer.pad_token_id,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{final_prompt}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN WRITTEN CODE:\n{code}')

**Working Good**

In [31]:
from datasets import load_dataset
dataset_4 = load_dataset("codeparrot/github-jupyter-code-to-text")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [32]:
dataset_4

DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'license', 'content'],
        num_rows: 47452
    })
    test: Dataset({
        features: ['repo_name', 'path', 'license', 'content'],
        num_rows: 11864
    })
})

In [33]:
print(dataset_4['train'][23]['path'])    # useful
print(dataset_4['train'][23]['repo_name'])
print(dataset_4['train'][23]['license'])

lucid_work/notebooks/feature_visualization.ipynb
davidparks21/qso_lya_detection_pipeline
mit


In [34]:
print(dataset_4['train'][263]['content'])

%run ../bst/bst.py
%load ../bst/bst.py

def height(node):
    # TODO: Implement me
    pass

"""
Explanation: <small><i>This notebook was prepared by Donne Martin. Source and license info is on GitHub.</i></small>
Challenge Notebook
Problem: Determine the height of a tree.

Constraints
Test Cases
Algorithm
Code
Unit Test
Solution Notebook

Constraints

Is this a binary tree?
Yes


Can we assume we already have a Node class with an insert method?
Yes



Test Cases

5 -> 1
5, 2, 8, 1, 3 -> 3

Algorithm
Refer to the Solution Notebook.  If you are stuck and need a hint, the solution notebook's algorithm discussion might be a good place to start.
Code
End of explanation
"""


# %load test_height.py
from nose.tools import assert_equal


class TestHeight(object):

    def test_height(self):
        root = Node(5)
        assert_equal(height(root), 1)
        insert(root, 2)
        insert(root, 8)
        insert(root, 1)
        insert(root, 3)
        assert_equal(height(root), 3)

        p

In [35]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
detect = fasttext.load_model(model_path)

In [36]:
def detect_lang(data):
    lines = data.split('\n')
    text = ''
    for i in range(len(lines)):
        if i < 4:
            text = text + lines[i]
        else:
            break
    lang = detect.predict(text)[0][0]
    if lang == '__label__eng_Latn':
        return 'en'
    else:
        return ''

In [37]:
def dataset_formation(data):
    add_prompt = '<SYSTEM_TASK:>\nGiven the following code description, write Python code to implement the functionality described below\n<END_TASK>\n<USER_TASK:>\nDescription:\n'
    s = data['content']
    dummy = data['content']
    occurrences = re.finditer('"""', s)
    # using reduce() to get start indices of all occurrences
    res = reduce(lambda x, y: x + [y.start()], occurrences, [])
    extracted_prompt_arr = []
    extracted_code = ''
    final_prompt = ''
    code = ''
    if len(res)%2 == 0:
        for i in range(int(len(res)/2)):
            extracted_prompt_arr.append(s[res[2*i] + 3 : res[2*i + 1]] + '\n')
            dummy = dummy.replace(s[res[2*i] : res[2*i + 1] + 3], '')
        final = ''
        extracted_code = dummy
        # LANGUAGE DETECTION  
        lang = detect_lang(extracted_prompt_arr[0])
        if lang == 'en':
            final_1, final_2 = '', ''
            for i in range(len(extracted_prompt_arr)):
                    exp = extracted_prompt_arr[i]
                    occurrences_1 = re.finditer('Explanation:', exp)
                    # using reduce() to get start indices of all occurrences
                    start = reduce(lambda x, y: x + [y.start()], occurrences_1, [])
                    occurrences_2 = re.finditer('End of explanation', exp)
                    end = reduce(lambda x, y: x + [y.start()], occurrences_2, [])
                    if len(start) != 0 and len(end) != 0:
                        extracted_exp = exp[start[0] + 12 : end[0]]
                        final_1 = final_1 + extracted_exp + '\n'
            final_2 = final_1
            occurrences_3 = re.finditer('<', final_1)
            start = reduce(lambda x, y: x + [y.start()], occurrences_3, [])   
            occurrences_4 = re.finditer('>', final_1)
            end = reduce(lambda x, y: x + [y.start()], occurrences_4, [])
            if len(start) == len(end) and len(start) != 0:
                for i in range(len(start)):
                    final_2 = final_2.replace(final_1[start[i] : end[i]], '')
            else:
                final_2 = final_1
            final_3 = final_2
            occurrences_5 = re.finditer('<', final_2)
            start = reduce(lambda x, y: x + [y.start()], occurrences_5, [])   
            occurrences_6 = re.finditer('>', final_2)
            end = reduce(lambda x, y: x + [y.start()], occurrences_6, [])
            if len(start) == len(end) and len(start) != 0:
                for i in range(len(start)):
                    final_3 = final_3.replace(final_2[start[i] : end[i]], '')
            else:
                final_3 = final_2
            final = final_3 
            if final != '':
                final_line = ''
                arr = final.split('\n')
                for i in range(len(arr)):
                    if i < 9:
                        final_line = final_line + arr[i] + ' '
                path = data['path'].split('/')
                func = path[-1].split('.')
                final_prompt = add_prompt + f'def {func[0]}():' + '\n\t' + f'"""{final_line}"""'   
                lines = extracted_code.split('\n')
                code = ''
                if len(lines) > 1:
                    for i in range(len(lines)):
                        if len(lines[i]) != 0:
                            if lines[i][0] != '#':
                                code = code + lines[i] + '\n'
                            else:
                                pass
                else:
                    code = extracted_code
    else:
        print('Error ^_^')
            
    return{
        'text_prompt' : final_prompt,
        'code_prompt' : code
    }   

In [38]:
prompted_dataset_4 = dataset_4.map(dataset_formation, batched = False, remove_columns = dataset_4['train'].column_names)

prompted_dataset_4

Parameter 'function'=<function dataset_formation at 0x7fceff46b1c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/47452 [00:00<?, ? examples/s]

Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^


Map:   0%|          | 0/11864 [00:00<?, ? examples/s]

Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^
Error ^_^


DatasetDict({
    train: Dataset({
        features: ['text_prompt', 'code_prompt'],
        num_rows: 47452
    })
    test: Dataset({
        features: ['text_prompt', 'code_prompt'],
        num_rows: 11864
    })
})

In [39]:
print(prompted_dataset_4['train'][14964]['text_prompt'])

<SYSTEM_TASK:>
Given the following code description, write Python code to implement the functionality described below
<END_TASK>
<USER_TASK:>
Description:
def student_intervention-checkpoint():
	""" Project 2: Supervised Learning Building a Student Intervention System 1. Classification vs Regression Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why? It is a classification problem. Because the output contains just 2 classes i.e. pass: yes or no.  2. Exploring the Data Let's go ahead and read in the student dataset first. To execute a code cell, click inside it and press Shift+Enter.  """


In [40]:
print(prompted_dataset_4['train'][6]['code_prompt'])

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import yaml
with open("data/swiss-reviews.txt", 'r') as fp:
    swiss_rev = fp.readlines()
len(swiss_rev)
swiss_rev[2]
def filter_helpful(line):
    l = line.rstrip('\n')
    l = yaml.load(l)
    if('helpful' in l.keys()):
        if(l['helpful'][1] >= 5):
            return True
        else:
            return False
    else:
        print("Review does not have helpful score key: "+line)
        return False
def get_helpful(data):
    res = []
    counter = 1
    i = 0
    for line in data:
        i += 1
        if(filter_helpful(line)):
            if(counter % 1000 == 0):
                print("Count "+str(counter)+" / "+str(i))
            counter += 1
            res.append(line)
    return res
swiss_reviews_helpful = get_helpful(swiss_rev)
len(swiss_reviews_helpful)
write_file = open('data/swiss-reviews-helpful-correct-bigger.txt', 'w')
for item in swiss_reviews_helpful:
  write_file.write

In [ ]:
final_prompt = prompted_dataset_4['test'][7114]['text_prompt']
code = prompted_dataset_4['test'][7114]['code_prompt']
inputs = tokenizer(final_prompt, return_tensors = 'pt').to('cuda')
generation_config = GenerationConfig(max_new_tokens=150, temperature= 1.5, do_sample=True, top_k = 5)

output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 150,
        generation_config=generation_config,
        pad_token_id = tokenizer.pad_token_id,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{final_prompt}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN WRITTEN CODE:\n{code}')

This dataset is OKAYISH so we will take 35% of this dataset

**Preparing final dataset**

In [195]:
# converting dataset_2 to pandas dataframe
import pandas as pd
df_1 = prompted_dataset_3['test'].to_pandas()

In [196]:
df_2 = prompted_dataset_2['train'].to_pandas()
df_3 = prompted_dataset_2['test'].to_pandas()
df_4 = prompted_dataset_2['validation'].to_pandas()

In [197]:
df_5 = prompted_dataset_4['train'].to_pandas()
df_6 = prompted_dataset_4['test'].to_pandas()

In [198]:
l = int(len(df_5)*0.25)
df_5 = df_5[:l]
df_6 = df_6[:l]

In [199]:
frames = [df_1, df_2,df_3, df_4, df_5, df_6]

df_final = pd.concat(frames, axis = 0, join = 'outer')
df_final.reset_index(inplace = True)
df_final.drop(['index'], axis = 1, inplace = True)
print(len(df_final))
df_final.head(5)

304542


,text_prompt,code_prompt
0,<SYSTEM_TASK:>\nGiven the following code descr...,"for idx, elem in enumerate(numbers):\n ..."
1,<SYSTEM_TASK:>\nGiven the following code descr...,result = []\n current_string = []\n ...
2,<SYSTEM_TASK:>\nGiven the following code descr...,return number % 1.0\n
3,<SYSTEM_TASK:>\nGiven the following code descr...,balance = 0\n\n for op in operations:\n...
4,<SYSTEM_TASK:>\nGiven the following code descr...,mean = sum(numbers) / len(numbers)\n re...


In [200]:
df_final.drop_duplicates(inplace = True)
df_final.dropna(inplace = True)
df_final.sample(frac = 1)
df_final.reset_index(inplace = True)
df_final.drop(['index'], axis = 1, inplace = True)

In [201]:
def detect_lang(data):
    lang = detect.predict(data)[0][0]
    if lang == '__label__eng_Latn':
        return 'en'
    else:
        return ''
    
    
for i in range(len(df_final)):
    prompt = df_final['text_prompt'][i]
    prompt = prompt.replace('\n', '')
    prompt = prompt[:2000]
    lang = detect_lang(prompt)
    if lang != 'en':
        df_final.drop([i], axis = 0, inplace = True)
    else:
        pass

In [207]:
print(len(df_final))

233208


In [209]:
split = 0.6
train_df = df_final[:int(len(df_final)*split)]
test_df = df_final[int(len(df_final)*0.6): int(len(df_final)*0.8)]
val_df = df_final[int(len(df_final)*0.8):]

print(len(val_df), len(df_final), len(test_df))

46642 233208 46642


In [210]:
train_df.reset_index(inplace = True)
test_df.reset_index(inplace = True)
val_df.reset_index(inplace = True)

In [211]:
train_df.drop(['index'], axis = 1, inplace = True)
test_df.drop(['index'], axis = 1, inplace = True)
val_df.drop(['index'], axis = 1, inplace = True)

/tmp/ipykernel_28/3316440186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(['index'], axis = 1, inplace = True)
/tmp/ipykernel_28/3316440186.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(['index'], axis = 1, inplace = True)
/tmp/ipykernel_28/3316440186.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df.drop(['index'], axis = 1, inplace = True)


In [212]:
print(None in train_df, None in test_df, None in val_df)

False False False


In [220]:
train_df.to_csv('train_data.csv', index = False, index_label=None)

In [221]:
test_df.to_csv('test_data.csv', index = False, index_label=None)

In [222]:
val_df.to_csv('validation_data.csv', index = False, index_label=None)